In [1]:
import os
os.environ['OMP_NUM_THREADS']='2'
os.environ['LD_LIBRARY_PATH']=''
os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [2]:
%cd /home/pz281@ad.eng.cam.ac.uk/mnt/PhD/Pro_Down_SR

/home/pz281@ad.eng.cam.ac.uk/mnt/PhD/Pro_Down_SR


In [3]:
from data_generation import *
from scipy.linalg import sqrtm
from downscaling import *
from utils import *
import random

/home/pz281@ad.eng.cam.ac.uk/anaconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%cd Matrix_Downsampling/

/home/pz281@ad.eng.cam.ac.uk/mnt/PhD/Pro_Down_SR/Matrix_Downsampling


## Langevin & Training Downscale Network

### Upscale By 4

In [10]:
N_low = 21
N_high = 81
scale = 4
a,b,c = 8,5,5

h_low = 1/(N_low-1)
x_low = np.arange(0,1.0001,h_low)
y_low = np.arange(0,1.0001,h_low)

h_high = 1/(N_high-1)
x_high = np.arange(0,1.0001,h_high)
y_high = np.arange(0,1.0001,h_high)

In [11]:
A_high = create_A(N_high)
A_low = create_A(N_low)

In [12]:
# Code downscaling matrix
H = np.zeros((N_low*N_low, N_high*N_high))

submatrix = np.zeros((N_low,N_high))
for i in range(N_low):
    submatrix[i,scale*i] = 1
    
for j in range(N_low):
    H[N_low*j:N_low*(j+1),N_high*scale*j:N_high*(scale*j+1)] = submatrix

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Parameters for RBF kernal
prior_sigma = 0.002
ll_sigma = 0.002

In [14]:
G = np.eye(N_high**2) * prior_sigma**2
G_inverse = np.eye(N_high**2) * (1/prior_sigma**2)

# Code downscaling matrix
H = np.zeros((N_low*N_low, N_high*N_high))

submatrix = np.zeros((N_low,N_high))
for i in range(N_low):
    submatrix[i,scale*i] = 1
    
for j in range(N_low):
    H[N_low*j:N_low*(j+1),N_high*scale*j:N_high*(scale*j+1)] = submatrix
    
# Turn matrices to tensors
G = torch.tensor(G).to(torch.float32).to(device)
G_inverse = torch.tensor(G_inverse).to(torch.float32).to(device)
H = torch.tensor(H).to(torch.float32).to(device)
A_high = torch.tensor(create_A(N_high)).to(torch.float32).to(device)
b_high = torch.tensor(create_forcing_term(N_high,a,b,c)).to(torch.float32).to(device)

# Store sparse matrices as sparse tensor
A_high = A_high.to_sparse()
H = H.to_sparse()
G = G.to_sparse()
G_inverse = G_inverse.to_sparse()
operator = torch.spmm(A_high.T,G_inverse).to(device)

In [17]:
dataset = DataFromH5File4("/home/pz281@ad.eng.cam.ac.uk/mnt/PhD/Pro_Down_SR/data/21_81_low_forcing.h5")

trainset = random.sample(range(0, 128), 100)
testset = [i for i in range(0,128) if i not in trainset]

In [18]:
def sample_data():
    coefficient = random.sample(trainset,1)[0]
    forcing = dataset[coefficient][0]
    lr = dataset[coefficient][1]
    
    return forcing, lr


def sample_p_0():
    # Randomly sampling for initialisation of the Langevin dynamics
    # prior = torch.randn(*[batch_size,1,20,20]).to(device)
    
    # Set the u_low_mean to the initialisation of the Langevin dynamics
    posterior_initial = torch.randn([N_high,N_high]).to(torch.float32)
    posterior_initial = torch.tensor(posterior_initial).to(device).to(torch.float32)
    
    return posterior_initial

    
def ula_posterior_preconditioner(z, b_high, x, G):
    """
    Langevin dynamics with preconditioner
    """
    z = z.clone().detach().requires_grad_(True)
    for i in range(K):
        # Grad log-likelihood
        observation = torch.spmm(H,z.reshape(N_high*N_high,1)).reshape(N_low,N_low)
        x_hat = observation + G(observation.reshape(1,N_low,N_low)).reshape(N_low,N_low)
        log_likelihood = (-1/(2*math.pow(ll_sigma, 2)) * torch.matmul((x-x_hat).reshape(1,N_low**2),(x-x_hat).reshape(N_low**2,1)))
        grad_ll = torch.autograd.grad(log_likelihood, z)[0]

        # Grad prior
        difference = torch.spmm(A_high,z.reshape(N_high*N_high,1)) - b_high.reshape(N_high**2,1)
        # log_prior = - 0.5 * difference.T @ G_inverse @ difference
        # grad_log_prior = torch.autograd.grad(log_prior, z)[0]
        grad_log_prior = (- torch.spmm(operator,difference)).reshape(N_high,N_high)
        
        # Random noise term
        W = torch.randn(*[N_high,N_high]).to(device)
        # random = torch.matmul(G_sqrt,W.reshape(N_high**2,1)).reshape(N_high,N_high)
        
        z = z + 0.5 * s ** 2 * grad_log_prior + 0.5 * s ** 2 * grad_ll + s * W
        # chains_evolution.append(z.cpu().data.numpy())   
           
    return z.detach()

In [18]:
# Train with sampled data
epoch_num = 1000
lr = 0.001
gamma = 0.1
step_size = 50
minimum_loss = float('inf')
loss_track = []

K = 4000
s = 0.0004

G = ResidualLearning()
G.apply(weights_init_xavier).to(device)
mse = nn.MSELoss(reduction='sum')
optG = torch.optim.Adam(G.parameters(), lr = lr, weight_decay=0, betas=(0.5, 0.999))
r_scheduleG = torch.optim.lr_scheduler.StepLR(optG, step_size=step_size, gamma=gamma)

# Logger info
dir_name = f'models/model3/21_81/lr{lr}_gamma{gamma}_stepsize{step_size}_K{K}'
makedir(dir_name)
logger = setup_logging('job0', dir_name, console=True)
logger.info(f'Training for {epoch_num} epoches and learning rate is {lr}')

for epoch in range(1, epoch_num+1):
    
    b_high, low_res = sample_data()
    b_high = torch.tensor(b_high).to(torch.float32).to(device)
    low_res = torch.tensor(low_res).to(torch.float32).to(device)
    
    posterior_initial = sample_p_0()
    posterior_final = ula_posterior_preconditioner(posterior_initial, b_high, low_res, G)

    optG.zero_grad()
    
    observation = torch.spmm(H,posterior_final.reshape(N_high*N_high,1)).reshape(1,N_low,N_low)
    out = G(observation.reshape(1,N_low,N_low)) + observation
    loss = mse(out,low_res.reshape(1,N_low,N_low))
        
    loss.backward()
    optG.step()
    
    if loss < minimum_loss:
        save_model(dir_name, epoch, 'best_model', r_scheduleG, G, optG)
        minimum_loss = loss
            
    if epoch%100 == 0:
        save_model(dir_name, epoch, 'model_epoch_{}'.format(epoch), r_scheduleG, G, optG)
    
    save_model(dir_name, epoch, 'current_epoch', r_scheduleG, G, optG)
    
    loss_track.append(loss.cpu().data.numpy())
    np.save(f'{dir_name}/chains/loss_curve.npy', np.array(loss_track))
    
    print("Epoch:", epoch, "Loss:", loss)

    r_scheduleG.step()

2024-06-02 23:09:17,954 : Training for 1000 epoches and learning rate is 0.005


/tmp/ipykernel_6282/2499179023.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  posterior_initial = torch.tensor(posterior_initial).to(device).to(torch.float32)


Epoch: 1 Loss: tensor(0.0067, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 2 Loss: tensor(0.0031, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 3 Loss: tensor(0.0070, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 4 Loss: tensor(0.0032, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 5 Loss: tensor(0.0054, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 6 Loss: tensor(0.0046, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 7 Loss: tensor(0.0026, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 8 Loss: tensor(0.0043, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 9 Loss: tensor(0.0039, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 10 Loss: tensor(0.0053, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 11 Loss: tensor(0.0055, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 12 Loss: tensor(0.0097, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 13 Loss: tensor(0.0038, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 14 Loss: tenso

KeyboardInterrupt: 

### Upscale By 6

In [5]:
N_low = 21
N_high = 121
scale = 6
a,b,c = 8,5,5

h_low = 1/(N_low-1)
x_low = np.arange(0,1.0001,h_low)
y_low = np.arange(0,1.0001,h_low)

h_high = 1/(N_high-1)
x_high = np.arange(0,1.0001,h_high)
y_high = np.arange(0,1.0001,h_high)

In [6]:
A_high = create_A(N_high)
A_low = create_A(N_low)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Parameters for RBF kernal
prior_sigma = 0.002
ll_sigma = 0.001

In [8]:
G = np.eye(N_high**2) * prior_sigma**2
G_inverse = np.eye(N_high**2) * (1/prior_sigma**2)

# Code downscaling matrix
H = np.zeros((N_low*N_low, N_high*N_high))

submatrix = np.zeros((N_low,N_high))
for i in range(N_low):
    submatrix[i,scale*i] = 1
    
for j in range(N_low):
    H[N_low*j:N_low*(j+1),N_high*scale*j:N_high*(scale*j+1)] = submatrix
    
# Turn matrices to tensors
G = torch.tensor(G).to(torch.float32).to(device)
G_inverse = torch.tensor(G_inverse).to(torch.float32).to(device)
H = torch.tensor(H).to(torch.float32).to(device)
A_high = torch.tensor(create_A(N_high)).to(torch.float32).to(device)
b_high = torch.tensor(create_forcing_term(N_high,a,b,c)).to(torch.float32).to(device)

# Store sparse matrices as sparse tensor
A_high = A_high.to_sparse()
H = H.to_sparse()
G = G.to_sparse()
G_inverse = G_inverse.to_sparse()
operator = torch.spmm(A_high.T,G_inverse).to(device)

In [9]:
dataset = DataFromH5File4("/home/pz281@ad.eng.cam.ac.uk/mnt/PhD/Pro_Down_SR/data/21_121_low_forcing.h5")

trainset = random.sample(range(0, 128), 100)
testset = [i for i in range(0,128) if i not in trainset]

In [10]:
def sample_data():
    coefficient = random.sample(trainset,1)[0]
    forcing = dataset[coefficient][0]
    lr = dataset[coefficient][1]
    
    return forcing, lr


def sample_p_0():
    # Randomly sampling for initialisation of the Langevin dynamics
    # prior = torch.randn(*[batch_size,1,20,20]).to(device)
    
    # Set the u_low_mean to the initialisation of the Langevin dynamics
    posterior_initial = torch.randn([N_high,N_high]).to(torch.float32)
    posterior_initial = torch.tensor(posterior_initial).to(device).to(torch.float32)
    
    return posterior_initial

    
def ula_posterior_preconditioner(z, b_high, x, G):
    """
    Langevin dynamics with preconditioner
    """
    z = z.clone().detach().requires_grad_(True)
    for i in range(K):
        # Grad log-likelihood
        observation = torch.spmm(H,z.reshape(N_high*N_high,1)).reshape(N_low,N_low)
        x_hat = observation + G(observation.reshape(1,N_low,N_low)).reshape(N_low,N_low)
        log_likelihood = (-1/(2*math.pow(ll_sigma, 2)) * torch.matmul((x-x_hat).reshape(1,N_low**2),(x-x_hat).reshape(N_low**2,1)))
        grad_ll = torch.autograd.grad(log_likelihood, z)[0]

        # Grad prior
        difference = torch.spmm(A_high,z.reshape(N_high*N_high,1)) - b_high.reshape(N_high**2,1)
        # log_prior = - 0.5 * difference.T @ G_inverse @ difference
        # grad_log_prior = torch.autograd.grad(log_prior, z)[0]
        grad_log_prior = (- torch.spmm(operator,difference)).reshape(N_high,N_high)
        
        # Random noise term
        W = torch.randn(*[N_high,N_high]).to(device)
        # random = torch.matmul(G_sqrt,W.reshape(N_high**2,1)).reshape(N_high,N_high)
        
        z = z + 0.5 * s ** 2 * grad_log_prior + 0.5 * s ** 2 * grad_ll + s * W
        # chains_evolution.append(z.cpu().data.numpy())   
           
    return z.detach()

In [17]:
# Train with sampled data
epoch_num = 1000
lr = 0.003
gamma = 0.5
step_size = 50
minimum_loss = float('inf')
loss_track = []

K = 9000
s = 0.0004

G = ResidualLearning()
G.apply(weights_init_xavier).to(device)
mse = nn.MSELoss(reduction='sum')
optG = torch.optim.Adam(G.parameters(), lr = lr, weight_decay=0, betas=(0.5, 0.999))
r_scheduleG = torch.optim.lr_scheduler.StepLR(optG, step_size=step_size, gamma=gamma)

# Logger info
dir_name = f'models/model3/21_121/2_lr{lr}_gamma{gamma}_stepsize{step_size}_K{K}_llsigma_{ll_sigma}_psigma_{prior_sigma}'
makedir(dir_name)
logger = setup_logging('job0', dir_name, console=True)
logger.info(f'Training for {epoch_num} epoches and learning rate is {lr}')

for epoch in range(1, epoch_num+1):
    
    b_high, low_res = sample_data()
    b_high = torch.tensor(b_high).to(torch.float32).to(device)
    low_res = torch.tensor(low_res).to(torch.float32).to(device)
    
    posterior_initial = sample_p_0()
    posterior_final = ula_posterior_preconditioner(posterior_initial, b_high, low_res, G)

    optG.zero_grad()
    
    observation = torch.spmm(H,posterior_final.reshape(N_high*N_high,1)).reshape(1,N_low,N_low)
    residual = low_res.reshape(1,N_low,N_low) - observation
    out = G(observation.reshape(1,N_low,N_low))
    loss = mse(out,residual)
        
    loss.backward()
    optG.step()
    
    if loss < minimum_loss:
        save_model(dir_name, epoch, 'best_model', r_scheduleG, G, optG)
        minimum_loss = loss
            
    if epoch%100 == 0:
        save_model(dir_name, epoch, 'model_epoch_{}'.format(epoch), r_scheduleG, G, optG)
    
    save_model(dir_name, epoch, 'current_epoch', r_scheduleG, G, optG)
    
    loss_track.append(loss.cpu().data.numpy())
    np.save(f'{dir_name}/chains/loss_curve.npy', np.array(loss_track))
    
    print("Epoch:", epoch, "Loss:", loss)

    r_scheduleG.step()

Output directory already exists
2024-06-11 13:39:55,799 : Training for 1000 epoches and learning rate is 0.003


/tmp/ipykernel_11851/232675414.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  posterior_initial = torch.tensor(posterior_initial).to(device).to(torch.float32)


Epoch: 1 Loss: tensor(0.0383, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 2 Loss: tensor(0.0117, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 3 Loss: tensor(0.0051, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 4 Loss: tensor(0.0030, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 5 Loss: tensor(0.0018, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 6 Loss: tensor(0.0022, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 7 Loss: tensor(0.0015, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 8 Loss: tensor(0.0015, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 9 Loss: tensor(0.0010, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 10 Loss: tensor(0.0010, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 11 Loss: tensor(0.0009, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 12 Loss: tensor(0.0009, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 13 Loss: tensor(0.0008, device='cuda:0', grad_fn=<MseLossBackward0>)
Epoch: 14 Loss: tenso

KeyboardInterrupt: 